# 非线性规划

约束条件是非凸的，所以不能使用cvxpy库了

In [ ]:
from scipy.optimize import minimize


# 示例约束非线性优化问题
def objective(x):
    return x[0] ** 2 + x[1] ** 2


def constraint1(x):
    return x[0] + x[1] - 1


def constraint2(x):
    # 这里可以返回一个数组，数组数越多，更能求解
    return x[0] ** 2 + x[1] ** 2 - 1


# 初始点和约束条件
x0 = [0, 1]
# eq   形式为 h(x) = 0
# ineq 形式为 h(x) >= 0
constraints = [
    {'type': 'eq', 'fun': constraint1},
    {'type': 'eq', 'fun': constraint2}
]

# 求解约束非线性优化问题
result = minimize(objective, x0, constraints=constraints, method='SLSQP')
print(f"Minimum value: {result.fun}")
print(f"x: {result.x}")

In [16]:
from gekko import GEKKO

# 初始化 gekko，不使用远程服务器
m = GEKKO(remote=False)
# m.options.SOLVER = 0 使用模型预测控制求解器，适用于动态优化和非线性预测控制问题。
# m.options.SOLVER = 1 使用 APOPT 求解器，默认选项，适用于大多数问题。整数
# m.options.SOLVER = 2 使用 BPOPT 求解器，适用于大型混合整数优化问题。
# m.options.SOLVER = 3 使用 IPOPT 求解器，适用于大型非线性问题，需要连接到互联网以下载和使用。

# 先清除之前的约束条件，每次solve()之后会清空约束条件，可以定义回调函数在每次solve()之后会执行
# m.clear_eqs()

m.options.MAX_ITER = 1000  # 设置最大迭代次数为 1000
m.options.EV_TYPE = 2  # 设置容许误差类型为 2

# 定义变量
x = [
    m.Var(value=1),  # 变量 x1，初始值为 1，下界为 0，上界为 5
    m.Var(value=5),  # 变量 x2，初始值为 5，下界为 0，上界为 5
]

# 定义目标函数
m.Minimize(x[0] ** 2 + x[1] ** 2)  # 最小化 x1^2 + x2^2

# 定义约束条件
m.Equations([
    x[0] ** 2 + x[1] ** 2 == 1,
    x[0] + x[1] == 1,
])

# 求解优化问题
m.solve(disp=False)

# 打印结果
print('最优解:')
print('x:', x)
print('目标函数值:', m.options.objfcnval)

最优解:
x: [[-1.9079532404e-11], [1.0]]
目标函数值: 1.0


# 选址问题

In [1]:
from gekko import Gekko

m = Gekko(remote=False)

A = [
    [5, 1],
    [2, 7],
]
P = [
    [1.25, 1.25],
    [8.75, 0.75],
    [0.50, 4.75],
    [5.75, 5.00],
    [3.00, 6.50],
    [7.25, 7.25],
]
V = [3, 5, 4, 7, 6, 11]

# 料数
X = [[m.Var(0, lb=0) for y in range(6)] for x in range(2)]


def distance(x, y):
    return ((x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2) ** 0.5


# 两个临时矿场到工地的距离，也是最终结果的系数
D = [[distance(x, y) for y in P] for x in A]

m.Equations([
    sum(X[0]) <= 20,
    sum(X[1]) <= 20,
    X[0][0] + X[1][0] == 3,
    X[0][1] + X[1][1] == 5,
    X[0][2] + X[1][2] == 4,
    X[0][3] + X[1][3] == 7,
    X[0][4] + X[1][4] == 6,
    X[0][5] + X[1][5] == 11,
])

m.Minimize(sum([X[i][j] * D[i][j] for i in range(2) for j in range(6)]))

# 求解优化问题
m.solve(disp=False)

# 打印结果
print('最优解:')
print('x:', X)
print('目标函数值:', m.options.objfcnval)

最优解:
x: [[[3.0000000092], [5.0000000096], [0.0], [7.0000000086], [0.0], [0.9999999798]], [[0.0], [0.0], [4.0000000067], [0.0], [6.0000000089], [10.00000002]]]
目标函数值: 135.2815419
